# Geospatial ML - Further Analysis - Part III


### Orignal Repo

is [here](https://github.com/Solve-Geosolutions/transform_2022)

### Original Data License

All data presented in this tutorial were derived from open data sets made available through [Mineral Resources Tasmania](https://www.mrt.tas.gov.au/) and [Geoscience Australia](https://www.ga.gov.au/).

**LICENSE CONDITIONS**

By exporting this data you accept and comply with the terms and conditions set out below:

[Creative Commons Attribution 3.0 Australia](https://creativecommons.org/licenses/by/3.0/au/)

<img src="assets/creative_commons_logo.png" />

# Why This Mining Geology Example?

This is a great example of how we need to account for [spatial autocorrelation](https://www.paulamoraga.com/book-spatial/spatial-autocorrelation.html#) (another reference [here](https://mgimond.github.io/Spatial/spatial-autocorrelation.html)) for machine learning projects. Rocks generally (but not always!) are not time dependent. I would not necessairly use the same methods for time dependent weather/climate datasets. 

## Roadmap

  1. Load and inspect data sets
  1. Combine data sets to build a labeled N<sub>pixel</sub>, N<sub>layers</sub> array for model training
      - inspect differences between proximal vs. distal to mineralisation pixels      
  1. Train a random GBDT classificer     
  1. Develop a checkerboard data selection procedure, train and evaluate models
      - discuss effects of spatially separated testing data
  2. Grid Search for the classifier
  3. Recursive Feature importance


 
---

You might need to add these two packages to your enviroment:

In [ ]:
!pip install rasterio

In [ ]:
!pip install imblearn

In [ ]:
# Import key packages
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd
import rasterio
from rasterio.features import rasterize
# Machine learning packages
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV 
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold, cross_val_score

from sklearn.metrics import f1_score, precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold

from CheckmateSample.generator import make_checkerboard

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance


print('XGboost version', xgb.__version__ )

In [ ]:
# set some plotting parameters 
mpl.rcParams.update({"axes.grid":True, "grid.color":"gray", "grid.linestyle":'--','figure.figsize':(10,10)})

## Pointing to our data directory

In [ ]:
data_dir = 'geodata/'

# set path to minoccs
point_fn = os.path.join(data_dir, 'sn_w_minoccs.gpkg')

# make a list of rasters in the data directory
geotiffs = [os.path.join(data_dir, x) for x in os.listdir(data_dir) if '.tif' in x]
point_fn, geotiffs

First cut at plotting, where do we have occurances

In [ ]:
df = gpd.read_file(point_fn)
df.plot()

### Reading in GeoTiffs

In [ ]:
# read geotiffs
data, names = [], []  # Lists to store data and corresponding file names
for fn in geotiffs:  # Loop through each GeoTIFF file
    with rasterio.open(fn, 'r') as src:  # Open GeoTIFF file for reading
        # read spatial information
        transform = src.transform  # Get affine transformation matrix
        region = (src.bounds[0], src.bounds[2], src.bounds[1], src.bounds[3])  # Get bounding box coordinates (left, bottom, right, top)
        # read band 1 data
        d = src.read(1)  # Read data from the first band
        nodata_mask = d == src.nodata  # Create a mask for NoData values
        d[nodata_mask] = np.nan  # Replace NoData values with NaN
        # append data to lists
        data.append(d)  # Append data to the list
        names.append(os.path.basename(fn).replace('.tif',''))  # Append file name to the list (without extension)

# stack list into 3D numpy array
data = np.stack(data)  # Stack the list of arrays into a 3D numpy array
data.shape, names  # Return the shape of the data array and the list of file names

In [ ]:
# plot the data
fig, axes = plt.subplots(3,3,figsize=(12,18))
for i, ax in enumerate(axes.flatten()):
    if i < data.shape[0]:
        ax.imshow(data[i], vmin=np.nanpercentile(data[i], 5), vmax=np.nanpercentile(data[i], 95), extent=region)
        ax.set(title=names[i])
        df.plot(ax=ax, marker='*', facecolor='w')
    else:
        ax.axis('off')
plt.show()

In [ ]:
# rasterize the point
geometry_generator = ((geom, 1) for geom in df.buffer(1000).geometry)
labels = rasterize(shapes=geometry_generator, out_shape=data[0].shape, fill=0, transform=transform).astype('float32')
labels[nodata_mask] = np.nan

## plt.imshow(labels)

In [ ]:
# Check dimensions
if np.shape(labels.flatten())[0] != np.shape(data.reshape((data.shape[0], data.shape[1] * data.shape[2])).T)[0]:
    raise ValueError("Labels and Data shapes (0th dimmension) do not match.")

X_pix = data.reshape((data.shape[0], data.shape[1] * data.shape[2])).T
y_pix = labels.flatten()

# remove nans
X = X_pix[~np.isnan(y_pix)]
y = y_pix[~np.isnan(y_pix)]

print(f"Shape of data after removing NaNs (X): {X.shape}")
print(f"Shape of labels after removing NaNs (y): {y.shape}")

## Train Models

In [ ]:
cpus = os.cpu_count()
print(f'The number of available CPUs is: {cpus}')

In this notebook, we will generally use 1 less. I have found that it does not slow it down a ton, but makes the user experiance generally much better. 

### Generate train and testing subsets
We will use the checkerboard example from the previous notebook

In [ ]:
# define a function to get probability map
def get_proba_map(X_pix, nodata_mask, model):
    """
    Generates a probability map from input features using a trained model.
    
    Parameters:
    X_pix (np.ndarray): A NumPy array containing the input features for prediction. Each row represents a pixel, and each column represents a feature.
    nodata_mask (np.ndarray): A boolean array with the same shape as the first dimension of X_pix. True values indicate pixels with no data (nodata).
    model (sklearn.base.BaseEstimator): A trained scikit-learn model that supports the predict_proba method.
    feature_names (list of str): A list of feature names corresponding to the columns in X_pix.
    
    Returns:
    np.ndarray: A 2D array with the same shape as nodata_mask, containing the predicted probabilities for each pixel. Pixels with no data are assigned NaN.
    """
    # Remove nulls by filtering out pixels where nodata_mask is True
    X = X_pix[np.invert(nodata_mask.flatten())]
    
    # Get predictions from the model (probability of the positive class)
    predictions = model.predict_proba(X)[:, 1]
    
    # Create an output array initialized to zeros, with the same shape as the flattened nodata_mask
    pred_ar = np.zeros(shape=nodata_mask.flatten().shape, dtype='float32')
    
    # Insert predictions into the output array at positions where nodata_mask is False
    pred_ar[np.invert(nodata_mask.flatten())] = predictions
    
    # Reshape the output array to match the original shape of nodata_mask
    pred_ar = pred_ar.reshape(nodata_mask.shape)
    
    # Assign NaN to positions where nodata_mask is True
    pred_ar[nodata_mask] = np.nan
    
    return pred_ar

### Recap questions:

What are the big differences between the first and second model?

Which one do you like more? 

Any downsides to this methodology?

# How to (try to) overcome spatial autocorrelation

In [ ]:
rus = RandomUnderSampler(random_state=32)

In [ ]:
# make checkerboard
checker = make_checkerboard(data[0].shape, (400,400))
checker[nodata_mask] = np.nan

#plot checkerboard
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(checker, extent=region)
df.plot(ax=ax, marker='*', facecolor='r')
plt.show()

In [ ]:
# data into checkers
X_check0 = X_pix[checker.flatten()==0]
y_check0 = y_pix[checker.flatten()==0]

X_check1 = X_pix[checker.flatten()==1]
y_check1 = y_pix[checker.flatten()==1]

# remove nans
X_check0 = X_check0[~np.isnan(y_check0)]
y_check0 = y_check0[~np.isnan(y_check0)]

X_check1 = X_check1[~np.isnan(y_check1)]
y_check1 = y_check1[~np.isnan(y_check1)]

# print some details
print ('Checker 0: X data array shape is {}, y labels array shape is {}'.format(X_check0.shape, y_check0.shape))
print ('Checker 1: X data array shape is {}, y labels array shape is {}'.format(X_check1.shape, y_check1.shape))

# run undersampling
X_check0, y_check0 = rus.fit_resample(X_check0, y_check0)
X_check1, y_check1 = rus.fit_resample(X_check1, y_check1)

# print some details
print ('Checker 0: X data array shape is {}, y labels array shape is {}'.format(X_check0.shape, y_check0.shape))
print ('Checker 1: X data array shape is {}, y labels array shape is {}'.format(X_check1.shape, y_check1.shape))

For this project, we will use th checkerboard method from the first notebook. Splitting up the labels would also be valid. 

In [ ]:
# fit some models
model0 = XGBClassifier(n_jobs=cpus-1)
model1 = XGBClassifier(n_jobs=cpus-1)

model0.fit(X_check0, y_check0)
model1.fit(X_check1, y_check1)

In [ ]:
# Generate probability maps
pred_ar0 = get_proba_map(X_pix, nodata_mask, model0)
pred_ar1 = get_proba_map(X_pix, nodata_mask, model1)

# Calculate the difference map
difference_map = pred_ar0 - pred_ar1

In [ ]:
# Plot probability maps
fig, ax = plt.subplots(1, 3, figsize=(18, 10))
for i, (ar, title) in enumerate(zip([pred_ar0, pred_ar1, difference_map], ['Checkerboard0', 'Checkerboard1', 'Difference'])):
    if title == 'Difference':
        im = ax[i].imshow(ar, extent=region)
    else:
        im = ax[i].imshow(ar, extent=region, vmin=0, vmax=1)
    ax[i].set_title(title)
    # df.plot(ax=ax[i], marker='*', facecolor='w')
    fig.colorbar(im, ax=ax[i], orientation='vertical', fraction=0.064, pad=0.04)

plt.tight_layout()
plt.show()

These results might look a little different than previously due to using XGBoost compared to RandomForest from scikit learn.

## Hyperparmeter Grid Search

Grid search for hyperparameters in machine learning is essential because it systematically explores a range of hyperparameter combinations to find the optimal set that improves model performance. Hyperparameters, such as learning rates, regularization strengths, and kernel parameters, significantly influence the model's learning process and predictive accuracy. A grid search helps ensure that we thoroughly investigate the parameter space, identifying the configuration that maximizes model performance while minimizing overfitting. This method, although computationally intensive, leads to more robust and reliable models by providing a structured approach to optimize their settings for the best possible results.

### What is a hyperparameter?

A hyperparameter is a configuration parameter used in machine learning models that is set before the training process begins and is not learned from the data. Unlike model parameters, which are internal variables learned from the data during training (like weights in a neural network), hyperparameters are external configurations that govern the training process itself and the model's architecture. They play a crucial role in shaping the behavior of the model and its learning capabilities.

#### A quick test, with no parameter tuning:

In [ ]:
xgb_clf_test = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf_test.fit(X_check0, y_check0)
preds = xgb_clf_test.predict(X_check1)
accuracy_test = accuracy_score(y_check1, preds)
print(f"Test accuracy: {round(accuracy_test, 3)}")

Notice we are using one checkerboard to score the other. This is different than earlier where we just looked at raw probabilities for the overall map.

#### Let's see if we can improve on that!

In [ ]:
# Initialize XGBoost classifier
xgb_clf0 = xgb.XGBClassifier(use_label_encoder=False, 
                             eval_metric='logloss')

We will use this param_grid again, so will not label it 0 or 1.

In [ ]:
param_grid = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.05, 0.01, 0.5],
    'n_estimators': [25, 50, 100, 200],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

In [ ]:
# Set up GridSearchCV
grid_search0 = GridSearchCV(estimator=xgb_clf0, 
                           param_grid=param_grid, 
                           scoring='accuracy', 
                           cv=5, 
                           verbose=1, 
                           n_jobs=7)

In [ ]:
grid_search0.fit(X_check0, y_check0)

In [ ]:
print(f"Best parameters found: {grid_search0.best_params_}")
print(f"Best cross-validation score: {grid_search0.best_score_}")

In [ ]:
# Evaluate on test set
best_model = grid_search0.best_estimator_
y_pred = best_model.predict(X_check1)
accuracy = accuracy_score(y_check1, y_pred)
print(f"Test accuracy: {round(accuracy, 4)}")

In [ ]:
print('improvement using grid search:', np.round(accuracy - accuracy_test,3))

#### Now we will do the same thing, but flip-flop the datasets

In [ ]:
xgb_clf_test1 = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf_test1.fit(X_check1, y_check1)
preds = xgb_clf_test1.predict(X_check0)
accuracy_test1 = accuracy_score(y_check0, preds)
print(f"Test accuracy: {round(accuracy_test1, 3)}")

In [ ]:
# Initialize XGBoost classifier
xgb_clf1 = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Set up GridSearchCV
grid_search1 = GridSearchCV(estimator=xgb_clf0, 
                           param_grid=param_grid, 
                           scoring='accuracy', 
                           cv=5, 
                           verbose=1, 
                           n_jobs=7)

In [ ]:
grid_search1.fit(X_check1, y_check1)

In [ ]:
print(f"Best parameters found: {grid_search1.best_params_}")
print(f"Best cross-validation score: {grid_search1.best_score_}")

In [ ]:
# Evaluate on test set
best_model1 = grid_search1.best_estimator_
y_pred1 = best_model1.predict(X_check0)
accuracy1 = accuracy_score(y_check0, y_pred1)
print(f"Test accuracy: {round(accuracy, 4)}")

Few follow up questions:
- Why is using the other checkerboard a good test?
- How different are the various hyperparameters from the two searches?
- Did the improvment suprise you from standard hyperparameters to best ones?

## Recursive Feature Elimination and Feature Importance

In [ ]:
X_train = pd.DataFrame(X_check0, columns=names)
X_test = pd.DataFrame(X_check1, columns=names)

y_train = pd.DataFrame(y_check0)
y_test = pd.DataFrame(y_check1)

In [ ]:
# Create an XGBoost classifier
model = XGBClassifier(max_depth=3, 
                      n_estimators=50,
                      learning_rate=0.01)

# Create an RFE object
rfe = RFE(estimator=model)

# Fit the RFE object to the training data
rfe.fit(X_train, y_train)

# Get the selected features
selected_features = rfe.get_support(indices=True)

# Train a new XGBoost classifier on the selected features
model = XGBClassifier()
model.fit(X_train.iloc[:, selected_features], y_train)

# Evaluate the model on the testing set
y_pred = model.predict(X_test.iloc[:, selected_features])
accuracy = accuracy_score(y_test, y_pred)

# Calculate the accuracy for different number of features
n_features = range(1, X_train.shape[1] + 1)
accuracies = []
for n in n_features:
    selected_features = rfe.get_support(indices=True)[:n]
    model = XGBClassifier()
    model.fit(X_train.iloc[:, selected_features], y_train)
    y_pred = model.predict(X_test.iloc[:, selected_features])
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)


In [ ]:
model.fit(X_train, y_train)

# Get feature importances and sort them
feature_importances = model.feature_importances_
sorted_idx = np.argsort(feature_importances)[::-1]
sorted_features = [X_train.columns[i] for i in sorted_idx]
sorted_importances = feature_importances[sorted_idx]

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_importances)), sorted_importances)
plt.yticks(range(len(sorted_importances)), sorted_features)
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.gca().invert_yaxis()  # Invert y-axis to display largest importance at the top
plt.show()

In [ ]:
# Plot the accuracy vs. number of features
plt.figure(figsize=(6, 4))
plt.plot(n_features, accuracies)
plt.scatter(n_features, accuracies, alpha=0.4)
plt.xlabel('Number of features')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Number of features for XGBoost classification with RFE')

# Set the x axis to every data point
plt.xticks(n_features)

# Flip the x-axis
plt.gca().invert_yaxis()

plt.show()

In [ ]:
# Create an XGBoost classifier for RFE
base_model = XGBClassifier(max_depth=3, 
                           n_estimators=50,
                           learning_rate=0.01)

# Perform RFE once to rank all features
rfe = RFE(estimator=base_model, n_features_to_select=1)
rfe.fit(X_train, y_train)

# Get the ranking of features
feature_ranking = rfe.ranking_

# Sort features by their RFE ranking
rfe_sorted_features = [x for _, x in sorted(zip(feature_ranking, X_train.columns))]

# Calculate the accuracy for different numbers of features
n_features = range(1, X_train.shape[1] + 1)
accuracies = []

print("Number of features | Accuracy | Features (RFE order)")
print("-" * 60)

for n in n_features:
    # Select top n features based on RFE ranking
    selected_features = rfe_sorted_features[:n]
    
    # Train a new XGBoost classifier on the selected features
    model = XGBClassifier()
    model.fit(X_train[selected_features], y_train)
    
    # Evaluate the model on the testing set
    y_pred = model.predict(X_test[selected_features])
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    
    # Print the current number of features, accuracy, and the features used
    print(f"{n:^17} | {accuracy:.4f} | {', '.join(selected_features)}")

# Train a final XGBoost model on all features
final_model = XGBClassifier()
final_model.fit(X_train, y_train)

# Get feature importances from the final XGBoost model
xgb_feature_importances = final_model.feature_importances_
xgb_sorted_idx = np.argsort(xgb_feature_importances)[::-1]
xgb_sorted_features = [X_train.columns[i] for i in xgb_sorted_idx]
xgb_sorted_importances = xgb_feature_importances[xgb_sorted_idx]

# Create subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 8))

# Plot RFE feature ranking
ax1.barh(range(len(rfe_sorted_features)), range(len(rfe_sorted_features), 0, -1))
ax1.set_yticks(range(len(rfe_sorted_features)))
ax1.set_yticklabels(rfe_sorted_features)
ax1.set_xlabel('RFE Ranking (lower is better)')
ax1.set_ylabel('Feature')
ax1.set_title('RFE Feature Ranking')
ax1.invert_yaxis()  # Invert y-axis to display best ranked at the top

# Plot XGBoost feature importance
ax2.barh(range(len(xgb_sorted_importances)), xgb_sorted_importances)
ax2.set_yticks(range(len(xgb_sorted_importances)))
ax2.set_yticklabels(xgb_sorted_features)
ax2.set_xlabel('Importance')
ax2.set_ylabel('Feature')
ax2.set_title('XGBoost Feature Importance (from final model)')
ax2.invert_yaxis()  # Invert y-axis to display largest importance at the top

# Plot accuracy vs number of features
ax3.plot(n_features, accuracies)
ax3.scatter(n_features, accuracies, alpha=0.4)
ax3.set_xlabel('Number of features')
ax3.set_ylabel('Accuracy')
ax3.set_title('Accuracy vs. Number of features (RFE order)')
ax3.set_xticks(n_features[::5])  # Show every 5th feature on x-axis for readability
ax3.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Print the order of feature selection by RFE
print("\nOrder of feature selection by RFE:")
for i, feature in enumerate(rfe_sorted_features, 1):
    print(f"{i}. {feature}")

# Print the order of feature importance by XGBoost
print("\nOrder of feature importance by XGBoost:")
for i, feature in enumerate(xgb_sorted_features, 1):
    print(f"{i}. {feature}")

#### Accuracy Discussion

Accuracy in scikit-learn has several limitations for mining prediction problems:

Class imbalance is common in mining datasets. Rare but critical events like high-grade ore deposits or geotechnical hazards may be overlooked by models that achieve high accuracy by always predicting the most common class.The fixed threshold in accuracy calculations doesn't reflect the complex trade-offs in mining decisions, which often vary based on factors like mineral prices, operational costs, and company risk tolerance. Accuracy lacks granularity, treating all incorrect predictions equally regardless of how close they are to the true value. In mineral exploration or grade estimation, predictions that are close to the true value can still be valuable, even if not exactly correct. Accuracy doesn't distinguish between different types of errors. False positives (predicting a viable deposit where there isn't one) and false negatives (missing an existing deposit) have different economic and operational consequences in mining, but accuracy treats them identically. For continuous variables common in mining (e.g., ore grade, geotechnical properties, processing recovery rates), binning into categories for accuracy calculation results in loss of valuable information. These limitations highlight why accuracy alone is often insufficient for evaluating predictive models in mining applications. More nuanced metrics and evaluation approaches are typically needed to capture the full value and implications of model predictions in this complex and high-stakes industry.

# AUC - ROC

In [ ]:
# Create an XGBoost classifier for RFE
base_model = XGBClassifier(max_depth=3, n_estimators=50, learning_rate=0.01)

# Perform RFE once to rank all features
rfe = RFE(estimator=base_model, n_features_to_select=1)
rfe.fit(X_train, y_train)

# Get the ranking of features
feature_ranking = rfe.ranking_

# Sort features by their RFE ranking
rfe_sorted_features = [x for _, x in sorted(zip(feature_ranking, X_train.columns))]

# Calculate the AUC-ROC for different numbers of features using cross-validation
n_features = range(1, X_train.shape[1] + 1)
auc_scores = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Number of features | AUC-ROC | Features (RFE order)")
print("-" * 60)

for n in n_features:
    # Select top n features based on RFE ranking
    selected_features = rfe_sorted_features[:n]
    
    # Perform cross-validation
    cv_scores = []
    for train_idx, val_idx in cv.split(X_train, y_train):
        X_cv_train, X_cv_val = X_train.iloc[train_idx][selected_features], X_train.iloc[val_idx][selected_features]
        y_cv_train, y_cv_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model = XGBClassifier()
        model.fit(X_cv_train, y_cv_train)
        y_pred_proba = model.predict_proba(X_cv_val)[:, 1]
        cv_scores.append(roc_auc_score(y_cv_val, y_pred_proba))
    
    mean_auc_score = np.mean(cv_scores)
    auc_scores.append(mean_auc_score)
    
    print(f"{n:^17} | {mean_auc_score:.4f} | {', '.join(selected_features)}")

# Create subplots in a 2x2 grid
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 20))

# Plot RFE feature ranking
ax1.barh(range(len(rfe_sorted_features)), range(len(rfe_sorted_features), 0, -1))
ax1.set_yticks(range(len(rfe_sorted_features)))
ax1.set_yticklabels(rfe_sorted_features)
ax1.set_xlabel('RFE Ranking (lower is better)')
ax1.set_ylabel('Feature')
ax1.set_title('RFE Feature Ranking')
ax1.invert_yaxis()  # Invert y-axis to display best ranked at the top

# Plot XGBoost feature importance
final_model = XGBClassifier()
final_model.fit(X_train, y_train)
xgb_feature_importances = final_model.feature_importances_
xgb_sorted_idx = np.argsort(xgb_feature_importances)[::-1]
xgb_sorted_features = [X_train.columns[i] for i in xgb_sorted_idx]
xgb_sorted_importances = xgb_feature_importances[xgb_sorted_idx]

ax2.barh(range(len(xgb_sorted_importances)), xgb_sorted_importances)
ax2.set_yticks(range(len(xgb_sorted_importances)))
ax2.set_yticklabels(xgb_sorted_features)
ax2.set_xlabel('Importance')
ax2.set_ylabel('Feature')
ax2.set_title('XGBoost Feature Importance (from final model)')
ax2.invert_yaxis()  # Invert y-axis to display largest importance at the top

# Plot AUC-ROC vs number of features with more x-axis ticks
ax3.plot(n_features, auc_scores)
ax3.scatter(n_features, auc_scores, alpha=0.4)
ax3.set_xlabel('Number of features')
ax3.set_ylabel('AUC-ROC')
ax3.set_title('AUC-ROC vs. Number of features (RFE order)')

# Set x-axis ticks to show every feature
ax3.set_xticks(n_features)
ax3.set_xticklabels(n_features)

# Add minor ticks for better readability
ax3.set_xticks(n_features, minor=True)
ax3.grid(True, linestyle='--', alpha=0.7, which='both')

# Plot ROC curves for specific numbers of features
ax4.plot([0, 1], [0, 1], linestyle='--', label='Random Classifier')
colors = ['red', 'green', 'blue', 'purple']
feature_subsets = [1, 3, 7, len(rfe_sorted_features)]

for n, color in zip(feature_subsets, colors):
    selected_features = rfe_sorted_features[:n]
    model = XGBClassifier()
    model.fit(X_train[selected_features], y_train)
    y_pred_proba = model.predict_proba(X_test[selected_features])[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    ax4.plot(fpr, tpr, color=color, label=f'{n} features (AUC = {auc:.3f})')

ax4.set_xlabel('False Positive Rate')
ax4.set_ylabel('True Positive Rate')
ax4.set_title('ROC Curves for Different Numbers of Features')
ax4.legend(loc='lower right')

plt.tight_layout()
plt.show()

# Print AUC-ROC scores for specific feature subsets
print("\nAUC-ROC scores for specific feature subsets:")
for n in feature_subsets:
    selected_features = rfe_sorted_features[:n]
    model = XGBClassifier()
    model.fit(X_train[selected_features], y_train)
    y_pred_proba = model.predict_proba(X_test[selected_features])[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)
    print(f"{n} features: {auc:.4f}")

# Print the order of feature selection by RFE
print("\nOrder of feature selection by RFE:")
for i, feature in enumerate(rfe_sorted_features, 1):
    print(f"{i}. {feature}")

# Print the order of feature importance by XGBoost
print("\nOrder of feature importance by XGBoost:")
for i, feature in enumerate(xgb_sorted_features, 1):
    print(f"{i}. {feature}")

In [ ]:
# Create an XGBoost classifier for RFE
base_model = XGBClassifier(max_depth=3, n_estimators=50, learning_rate=0.01)

# Perform RFE once to rank all features
rfe = RFE(estimator=base_model, n_features_to_select=1)
rfe.fit(X_train, y_train)

# Get the ranking of features
feature_ranking = rfe.ranking_

# Sort features by their RFE ranking
rfe_sorted_features = [x for _, x in sorted(zip(feature_ranking, X_train.columns))]

# Calculate the F1 score for different numbers of features using cross-validation
n_features = range(1, X_train.shape[1] + 1)
f1_scores = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Number of features | F1 Score | Features (RFE order)")
print("-" * 60)

for n in n_features:
    # Select top n features based on RFE ranking
    selected_features = rfe_sorted_features[:n]
    
    # Perform cross-validation
    cv_scores = []
    for train_idx, val_idx in cv.split(X_train, y_train):
        X_cv_train, X_cv_val = X_train.iloc[train_idx][selected_features], X_train.iloc[val_idx][selected_features]
        y_cv_train, y_cv_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model = XGBClassifier()
        model.fit(X_cv_train, y_cv_train)
        y_pred = model.predict(X_cv_val)
        cv_scores.append(f1_score(y_cv_val, y_pred))
    
    mean_f1_score = np.mean(cv_scores)
    f1_scores.append(mean_f1_score)
    
    print(f"{n:^17} | {mean_f1_score:.4f} | {', '.join(selected_features)}")

# Create subplots in a 2x2 grid
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 20))

# Plot RFE feature ranking
ax1.barh(range(len(rfe_sorted_features)), range(len(rfe_sorted_features), 0, -1))
ax1.set_yticks(range(len(rfe_sorted_features)))
ax1.set_yticklabels(rfe_sorted_features)
ax1.set_xlabel('RFE Ranking (lower is better)')
ax1.set_ylabel('Feature')
ax1.set_title('RFE Feature Ranking')
ax1.invert_yaxis()  # Invert y-axis to display best ranked at the top

# Plot XGBoost feature importance
final_model = XGBClassifier()
final_model.fit(X_train, y_train)
xgb_feature_importances = final_model.feature_importances_
xgb_sorted_idx = np.argsort(xgb_feature_importances)[::-1]
xgb_sorted_features = [X_train.columns[i] for i in xgb_sorted_idx]
xgb_sorted_importances = xgb_feature_importances[xgb_sorted_idx]

ax2.barh(range(len(xgb_sorted_importances)), xgb_sorted_importances)
ax2.set_yticks(range(len(xgb_sorted_importances)))
ax2.set_yticklabels(xgb_sorted_features)
ax2.set_xlabel('Importance')
ax2.set_ylabel('Feature')
ax2.set_title('XGBoost Feature Importance (from final model)')
ax2.invert_yaxis()  # Invert y-axis to display largest importance at the top

# Plot F1 score vs number of features with more x-axis ticks
ax3.plot(n_features, f1_scores)
ax3.scatter(n_features, f1_scores, alpha=0.4)
ax3.set_xlabel('Number of features')
ax3.set_ylabel('F1 Score')
ax3.set_title('F1 Score vs. Number of features (RFE order)')

# Set x-axis ticks to show every feature
ax3.set_xticks(n_features)
ax3.set_xticklabels(n_features)

# Add minor ticks for better readability
ax3.set_xticks(n_features, minor=True)
ax3.grid(True, linestyle='--', alpha=0.7, which='both')

# Plot Precision-Recall curves for specific numbers of features
colors = ['red', 'green', 'blue', 'purple']
total_features = len(rfe_sorted_features)
feature_subsets = [1, total_features // 2, total_features - 1, total_features]

for n, color in zip(feature_subsets, colors):
    selected_features = rfe_sorted_features[:n]
    model = XGBClassifier()
    model.fit(X_train[selected_features], y_train)
    y_pred_proba = model.predict_proba(X_test[selected_features])[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    f1 = f1_score(y_test, model.predict(X_test[selected_features]))
    ax4.plot(recall, precision, color=color, label=f'{n} features (F1 = {f1:.3f})')

ax4.set_xlabel('Recall')
ax4.set_ylabel('Precision')
ax4.set_title('Precision-Recall Curves for Different Numbers of Features')
ax4.legend(loc='lower left')

plt.tight_layout()
plt.show()

# Print F1 scores for specific feature subsets
print("\nF1 scores for specific feature subsets:")
for n in feature_subsets:
    selected_features = rfe_sorted_features[:n]
    model = XGBClassifier()
    model.fit(X_train[selected_features], y_train)
    y_pred = model.predict(X_test[selected_features])
    f1 = f1_score(y_test, y_pred)
    print(f"{n} features: {f1:.4f}")

# Print the order of feature selection by RFE
print("\nOrder of feature selection by RFE:")
for i, feature in enumerate(rfe_sorted_features, 1):
    print(f"{i}. {feature}")

# Print the order of feature importance by XGBoost
print("\nOrder of feature importance by XGBoost:")
for i, feature in enumerate(xgb_sorted_features, 1):
    print(f"{i}. {feature}")

In [ ]:
# Create an XGBoost classifier for RFE
base_model = XGBClassifier(max_depth=3, n_estimators=50, learning_rate=0.01)

# Perform RFE once to rank all features
rfe = RFE(estimator=base_model, n_features_to_select=1)
rfe.fit(X_train, y_train)

# Get the ranking of features
feature_ranking = rfe.ranking_

# Sort features by their RFE ranking
rfe_sorted_features = [x for _, x in sorted(zip(feature_ranking, X_train.columns))]

# Calculate the F1 score for different numbers of features using cross-validation
n_features = range(1, X_train.shape[1] + 1)
f1_scores = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Number of features | F1 Score (CV) | Features (RFE order)")
print("-" * 70)

for n in n_features:
    # Select top n features based on RFE ranking
    selected_features = rfe_sorted_features[:n]
    
    # Perform cross-validation
    cv_scores = cross_val_score(XGBClassifier(), X_train[selected_features], y_train, 
                                cv=cv, scoring='f1')
    mean_f1_score = np.mean(cv_scores)
    f1_scores.append(mean_f1_score)
    
    print(f"{n:^17} | {mean_f1_score:.4f} | {', '.join(selected_features)}")

# Create subplots in a 2x2 grid
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 20))

# Plot RFE feature ranking
ax1.barh(range(len(rfe_sorted_features)), range(len(rfe_sorted_features), 0, -1))
ax1.set_yticks(range(len(rfe_sorted_features)))
ax1.set_yticklabels(rfe_sorted_features)
ax1.set_xlabel('RFE Ranking (lower is better)')
ax1.set_ylabel('Feature')
ax1.set_title('RFE Feature Ranking')
ax1.invert_yaxis()  # Invert y-axis to display best ranked at the top

# Plot XGBoost feature importance
final_model = XGBClassifier()
final_model.fit(X_train, y_train)
xgb_feature_importances = final_model.feature_importances_
xgb_sorted_idx = np.argsort(xgb_feature_importances)[::-1]
xgb_sorted_features = [X_train.columns[i] for i in xgb_sorted_idx]
xgb_sorted_importances = xgb_feature_importances[xgb_sorted_idx]

ax2.barh(range(len(xgb_sorted_importances)), xgb_sorted_importances)
ax2.set_yticks(range(len(xgb_sorted_importances)))
ax2.set_yticklabels(xgb_sorted_features)
ax2.set_xlabel('Importance')
ax2.set_ylabel('Feature')
ax2.set_title('XGBoost Feature Importance (from final model)')
ax2.invert_yaxis()  # Invert y-axis to display largest importance at the top

# Plot F1 score vs number of features with more x-axis ticks
ax3.plot(n_features, f1_scores)
ax3.scatter(n_features, f1_scores, alpha=0.4)
ax3.set_xlabel('Number of features')
ax3.set_ylabel('F1 Score (Cross-validated)')
ax3.set_title('F1 Score vs. Number of features (RFE order)')

# Set x-axis ticks to show every feature
ax3.set_xticks(n_features)
ax3.set_xticklabels(n_features)

# Add minor ticks for better readability
ax3.set_xticks(n_features, minor=True)
ax3.invert_xaxis()
ax3.grid(True, linestyle='--', alpha=0.7, which='both')

# Plot Precision-Recall curves for specific numbers of features
colors = ['red', 'green', 'blue', 'purple', 'orange']
total_features = len(rfe_sorted_features)
feature_subsets = [1, 3, total_features // 2, total_features - 1, total_features]

print("\nDetailed F1 scores for specific feature subsets:")
print("Subset | Train F1 (CV) | Test F1 | Features")
print("-" * 70)

for n, color in zip(feature_subsets, colors):
    selected_features = rfe_sorted_features[:n]
    
    # Cross-validation on training set
    cv_scores = cross_val_score(XGBClassifier(), X_train[selected_features], y_train, 
                                cv=5, scoring='f1')
    mean_cv_f1 = np.mean(cv_scores)
    
    # Fit on entire training set and evaluate on test set
    model = XGBClassifier()
    model.fit(X_train[selected_features], y_train)
    y_pred = model.predict(X_test[selected_features])
    test_f1 = f1_score(y_test, y_pred)
    
    # Plot Precision-Recall curve
    y_pred_proba = model.predict_proba(X_test[selected_features])[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    ax4.plot(recall, precision, color=color, 
             label=f'{n} features (Test F1 = {test_f1:.3f})')
    
    print(f"{n:^6} | {mean_cv_f1:.4f} | {test_f1:.4f} | {', '.join(selected_features)}")

ax4.set_xlabel('Recall')
ax4.set_ylabel('Precision')
ax4.set_title('Precision-Recall Curves for Different Numbers of Features')
ax4.legend(loc='lower left')

plt.tight_layout